In [1]:
!pip install ir-datasets

     |████████████████████████████████| 127 kB 5.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.9
    Uninstalling urllib3-1.26.9:
      Successfully uninstalled urllib3-1.26.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
elastic-transport 8.1.2 requires urllib3<2,>=1.26.2, but you have urllib3 1.25.11 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
!pip install elasticsearch

  Using cached urllib3-1.26.9-py2.py3-none-any.whl (138 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
from datetime import datetime
import csv
from elasticsearch import Elasticsearch, helpers
import ir_datasets
import pandas as pd


dataset = ir_datasets.load("clinicaltrials/2021")

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [15]:
df = pd.DataFrame(dataset.docs_iter())

In [16]:
df

,doc_id,title,condition,summary,detailed_description,eligibility
0,NCT00000102,Congenital Adrenal Hyperplasia: Calcium Channe...,,\n \n This study will test the ability...,\n \n This protocol is designed to ass...,\n \n Inclusion Criteria:\r\n\r\n ...
1,NCT00000104,Does Lead Burden Alter Neuropsychological Deve...,,\n \n Inner city children are at an in...,,\n \n Inclusion Criteria:\r\n\r\n ...
2,NCT00000105,Vaccination With Tetanus and KLH to Assess Imm...,,\n \n The purpose of this study is to ...,\n \n Patients will receive each vacci...,\n \n Inclusion Criteria:\r\n\r\n ...
3,NCT00000106,41.8 Degree Centigrade Whole Body Hyperthermia...,,\n \n Recently a non-toxic system for ...,,\n \n Inclusion Criteria:\r\n\r\n ...
4,NCT00000107,Body Water Content in Cyanotic Congenital Hear...,,\n \n Adults with cyanotic congenital ...,,\n \n Inclusion Criteria:\r\n\r\n ...
...,...,...,...,...,...,...
375575,NCT04862260,Cholesterol Disruption in Combination With FOL...,,\n \n Cardiovascular diseases and canc...,,\n \n Inclusion Criteria:\r\n\r\n ...
375576,NCT04862273,Native T1 CMR Imaging for Diagnosis of Cardiac...,,\n \n The study aims to test the diagn...,\n \n Cardiac amyloidosis (CA) is an i...,\n \n Inclusion Criteria:\r\n\r\n ...
375577,NCT04862286,Study to Evaluate Adverse Events and Change in...,,"\n \n Psoriasis is a chronic, systemic...",,\n \n Inclusion Criteria:\r\n\r\n ...
375578,NCT04862299,Pelvic-perineal Disorders in Women With Sphinc...,,"\n \n According to INSEE, in 2016, in ...",,\n \n Inclusion Criteria:\r\n\r\n ...


In [17]:
df["content"] = df['title'].astype(str) +" "+ df["summary"].astype(str) + " " + df['detailed_description'].astype(str) + " " + df['eligibility']

In [18]:
df["content"][0]

'Congenital Adrenal Hyperplasia: Calcium Channels as Therapeutic Targets \n    \n      This study will test the ability of extended release nifedipine (Procardia XL), a blood\r\n      pressure medication, to permit a decrease in the dose of glucocorticoid medication children\r\n      take to treat congenital adrenal hyperplasia (CAH).\r\n    \n   \n    \n      This protocol is designed to assess both acute and chronic effects of the calcium channel\r\n      antagonist, nifedipine, on the hypothalamic-pituitary-adrenal axis in patients with\r\n      congenital adrenal hyperplasia. The multicenter trial is composed of two phases and will\r\n      involve a double-blind, placebo-controlled parallel design. The goal of Phase I is to examine\r\n      the ability of nifedipine vs. placebo to decrease adrenocorticotropic hormone (ACTH) levels,\r\n      as well as to begin to assess the dose-dependency of nifedipine effects. The goal of Phase II\r\n      is to evaluate the long-term effects of

In [21]:
df["content"] = df["content"].replace('\n','', regex=True)
df['content'] = df["content"].replace('\r','', regex=True)

In [22]:
df["content"][0]

'Congenital Adrenal Hyperplasia: Calcium Channels as Therapeutic Targets           This study will test the ability of extended release nifedipine (Procardia XL), a blood      pressure medication, to permit a decrease in the dose of glucocorticoid medication children      take to treat congenital adrenal hyperplasia (CAH).                 This protocol is designed to assess both acute and chronic effects of the calcium channel      antagonist, nifedipine, on the hypothalamic-pituitary-adrenal axis in patients with      congenital adrenal hyperplasia. The multicenter trial is composed of two phases and will      involve a double-blind, placebo-controlled parallel design. The goal of Phase I is to examine      the ability of nifedipine vs. placebo to decrease adrenocorticotropic hormone (ACTH) levels,      as well as to begin to assess the dose-dependency of nifedipine effects. The goal of Phase II      is to evaluate the long-term effects of nifedipine; that is, can attenuation of ACTH 

In [27]:
df

,doc_id,content
0,NCT00000102,Congenital Adrenal Hyperplasia: Calcium Channe...
1,NCT00000104,Does Lead Burden Alter Neuropsychological Deve...
2,NCT00000105,Vaccination With Tetanus and KLH to Assess Imm...
3,NCT00000106,41.8 Degree Centigrade Whole Body Hyperthermia...
4,NCT00000107,Body Water Content in Cyanotic Congenital Hear...
...,...,...
375575,NCT04862260,Cholesterol Disruption in Combination With FOL...
375576,NCT04862273,Native T1 CMR Imaging for Diagnosis of Cardiac...
375577,NCT04862286,Study to Evaluate Adverse Events and Change in...
375578,NCT04862299,Pelvic-perineal Disorders in Women With Sphinc...
